# Monitor capacity metrics
When your organization uses Microsoft Fabric, the license used to purchase the service determines the **_capacity_** available. A capacity is a pool of resources that you can use to implement Fabric capabilities.

The cost of using Fabric is based on **_capacity units (CUs)_**. Each action you perform in a Fabric resource can consume CUs, for which your organization is billed. It's therefore important to be able to monitor capacity usage to plan and manage costs. In data warehouse workloads, CUs are consumed by data read and write activities, so queries in your data warehouse and the underlying file operations to OneLake storage are a significant factor in the cost of your Fabric analytics solution.

## Using the Microsoft Fabric Capacity Metrics app
**The Microsoft Fabric Capacity Metrics app** is an app that an administrator can install in a Fabric environment and use to **_monitor capacity utilization_**. To monitor capacity utilization related to data warehousing, you can filter the interface to show only **warehouse** activity, like this:

<img src="../images/04_Implement a data warehouse with Microsoft Fabric/03/fabric-capacity-metrics-app.gif" alt="Fabric Capacity Metrics app" style="border: 2px solid black; border-radius: 10px;">

By using the Fabric Capacity Metrics app, you can observe capacity utilization trends to determine what processes are consuming CUs in your Fabric environment and whether any **_throttling_** is occurring (which indicates that your processes require more capacity than is available within the constraints of your purchased capacity license). With this information, you can optimize your capacity license for your needs.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> For more information about Microsoft Fabric Capacity Metrics app, refer to [Billing and utilization reporting in Synapse Data Warehouse](https://learn.microsoft.com/en-us/fabric/data-warehouse/usage-reporting) in the Microsoft Fabric documentation.

# Monitor current activity
You can use dynamic management views (DMVs) to retrieve information about the current state of the data warehouse. Specifically, Microsoft Fabric data warehouses include the following DMVs:

- **`sys.dm_exec_connections`**: Returns information about data warehouse connections.
- **`sys.dm_exec_sessions`**: Returns information about authenticated sessions.
- **`sys.dm_exec_requests`**: Returns information about active requests.

The schema of these tables is shown here:

<img src="../images/04_Implement a data warehouse with Microsoft Fabric/03/dynamic-management-views.png" alt="Diagram of dynamic management views." style="border: 2px solid black; border-radius: 10px;">

## Querying DMVs
You can retrieve detailed information about current activities in the data warehouse by querying the **`dm_exec_*`** DMVs. For example, consider the following query:

In [0]:
%sql
SELECT sessions.session_id, sessions.login_name,
    connections.client_net_address,
    requests.command, requests.start_time, requests.total_elapsed_time
FROM sys.dm_exec_connections AS connections
INNER JOIN sys.dm_exec_sessions AS sessions
    ON connections.session_id=sessions.session_id
INNER JOIN sys.dm_exec_requests AS requests
    ON requests.session_id = sessions.session_id
WHERE requests.status = 'running'
    AND requests.database_id = DB_ID()
ORDER BY requests.total_elapsed_time DESC

This query returns details about the active requests in the current database, ordered by the duration for which they have been executing; which may be useful to identify long-running queries that could benefit from optimization. An example result set from the query is shown here:

| session_id | login_name | client_net_address | command | start_time | total_elapsed_time |
|-- | -- | -- | -- | -- | -- |
| 60 | fred@contoso.com | 10.23.139.162 | SELECT | 2023-12-07T14:56:41.3530000 | 57266 |
| 126 | nandita@contoso.com | 10.23.137.98 | SELECT | 2023-12-07T14:57:22.7800000 | 15840 |
| 137 | zoe@contoso.com | 10.23.119.171 | SELECT | 2023-12-07T14:57:38.6070000 | 4 |

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> For more information about using DMVs, refer to [Monitor connections, sessions, and requests using DMVs](https://learn.microsoft.com/en-us/fabric/data-warehouse/monitor-using-dmv) in the Microsoft Fabric documentation.

# Monitor queries
Microsoft Fabric data warehouses include **_query insights_** feature that provides historical, aggregated information about the queries that have been run; enabling you to identify frequently used or long-running queries, and helping you analyze and tune query performance.

Query insights are provided through the following views:

- **`queryinsights.exec_requests_history`**: Details of each completed SQL query.
- **`queryinsights.long_running_queries`**: Details of query execution time.
- **`queryinsights.frequently_run_queries`**: Details of frequently run queries.
The schema for these tables is shown here:

<img src="../images/04_Implement a data warehouse with Microsoft Fabric/03/query-insights.png" alt="Query insights views" style="border: 2px solid black; border-radius: 10px;">

## Retrieving query insights
The query insights views are a useful source of information about the queries that are being run in your data warehouse.

### exec_requests_history

In [0]:
%sql
SELECT start_time, login_name, command
FROM queryinsights.exec_requests_history 
WHERE start_time >= DATEADD(MINUTE, -60, GETUTCDATE())

This query uses the **`queryinsights.exec_requests_history`** view to identify queries that were run in the previous hour.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> Depending on the concurrent workloads, queries may take up to 15 minutes to be reflected in the query insights views.

### long_running_queries

You can get details of long-running queries from the **`queryinsights.long_running_queries`** view like this:

In [0]:
%sql
SELECT last_run_command, number_of_runs, median_total_elapsed_time_ms, last_run_start_time
FROM queryinsights.long_running_queries
WHERE number_of_runs > 1
ORDER BY median_total_elapsed_time_ms DESC;

This query identifies long-running SQL commands that have been used more than once and returns them in descending order of their median time to complete.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> To enable the views to provide aggregated metrics, queries with predicates are parameterized and considered the same query if the parameterized statements are an exact match. For example, the following queries would be considered to be the same command:

`SELECT * FROM sales WHERE orderdate > '01/01/2023'`

`SELECT * FROM sales WHERE orderdate > '12/31/2021'`

### frequently_run_queries

To find commonly used queries, you can use the queryinsights.**`frequently_run_queries`** view, like this:

In [0]:
%sql
SELECT last_run_command, number_of_runs, number_of_successful_runs, number_of_failed_runs
FROM queryinsights.frequently_run_queries
ORDER BY number_of_runs DESC;

This query returns details of successful and failed runs for frequently run commands.

<img src="https://files.training.databricks.com/images/icon_note_32.png" alt="Note"> For more information about using query insights refer to [Query insights in Fabric data warehousing](https://learn.microsoft.com/en-us/fabric/data-warehouse/query-insights) in the Microsoft Fabric documentation.